# P ALS

In [9]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [10]:


import pandas as pd

with open('u.data', 'r') as file:
    # Read the lines of the file
    lines = file.readlines()

# Process the lines to create a list of tuples
data = []
for line in lines:
    parts = line.strip().split('\t')
    user_id = int(parts[0])
    item_id = int(parts[1])
    rating = int(parts[2])
    # Skip the timestamp (parts[3])
    data.append((user_id, item_id, rating))

# Create a DataFrame from the list of tuples
df = pd.DataFrame(data, columns=['userId', 'movieId', 'rating'])

# Print the first few rows of the DataFrame

df['movieId'].max()
df['movieId'].min()

temp = 1
userItem  = np.zeros((df['userId'].max()+1,df['movieId'].max()+1))
print(userItem )
print(userItem.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(944, 1683)


In [11]:
for i in range (len(df['userId'])):
  userItem[df.userId[i]][df.movieId[i]] = df.rating[i]

print(userItem)
print(userItem.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 5. 3. ... 0. 0. 0.]
 [0. 4. 0. ... 0. 0. 0.]
 ...
 [0. 5. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 5. ... 0. 0. 0.]]
(944, 1683)


In [12]:
R = userItem[1:]
print(R)
R = R[:, 1:]
print(R)

ep = [2, 4, 6, 8, 10, 12, 14, 16, 18, 20]

[[0. 5. 3. ... 0. 0. 0.]
 [0. 4. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 5. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 5. ... 0. 0. 0.]]
[[5. 3. 4. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [5. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 5. 0. ... 0. 0. 0.]]


In [13]:
A = R>0.5; A[A == True] = 1; A[A == False]   = 0; A = A.astype(np.float64, copy=False)

    

In [41]:
def find_element(matrix, j, element):
    # Find indices where element equals 5 in column j
    indices = np.where(matrix[:, j] == element)[0]
    return indices

In [58]:
r = find_element(R,143, 5)
print(r)

[  6   7  15  21  37  50  54  55  94 124 129 136 176 212 221 250 266 275
 290 291 302 325 331 341 346 351 373 378 393 397 400 415 420 435 446 449
 456 465 467 477 486 550 558 561 591 618 621 628 637 653 660 714 737 745
 748 795 805 822 830 850 863 879 891 900 901 906 912]


In [64]:
print(Rcap[50][143])

4.65871180936206


In [14]:
import matplotlib.pyplot as plt
import math



def runALS(A, R, n_factors, n_iterations, lambda_):
    '''
    Runs Alternating Least Squares algorithm in order to calculate matrix.

    :param A: User-Item Matrix with ratings
    :param R: User-Item Matrix with 1 if there is a rating or 0 if not
    :param n_factors: How many factors each of user and item matrix will consider
    :param n_iterations: How many times to run algorithm
    :param lambda_: Regularization parameter
    :return:
    '''
    print ("Initiating ")
    lambda_ = 0.1; n_factors = 3; n, m = A.shape; n_iterations = 20
    Users = 5 * np.random.rand(n, n_factors)
    Items = 5 * np.random.rand(n_factors, m)


    def get_error(A, Users, Items, R):
        # This calculates the MSE of nonzero elements
        return math.sqrt(np.sum((R * (A - np.dot(Users, Items))) ** 2) / np.sum(R))

    RMSE_List = []

    print ("Starting Iterations")
    for iter in range(n_iterations):
        for i, Ri in enumerate(R):
            Users[i] = np.linalg.solve(np.dot(Items, np.dot(np.diag(Ri), Items.T)) + lambda_ * np.eye(n_factors),
                                       np.dot(Items, np.dot(np.diag(Ri), A[i].T))).T
#             print('Users[i]',Users[i])
        print ("Error after solving for User Matrix:", get_error(A, Users, Items, R))

        for j, Rj in enumerate(R.T):
            Items[:,j] = np.linalg.solve(np.dot(Users.T, np.dot(np.diag(Rj), Users)) + lambda_ * np.eye(n_factors),
                                     np.dot(Users.T, np.dot(np.diag(Rj), A[:, j])))
        print ("Error after solving for Item Matrix:", get_error(A, Users, Items, R))

        RMSE_List.append(get_error(A, Users, Items, R))
        print ('%sth iteration is complete...' % iter)
        
    
    return Users,Items,RMSE_List.pop()
        

   

In [15]:
# import numpy as np
# R = np.array([[1,0,2,1],[4,5,1,0],[4,5,0,5],[4,1,0,6],[4,0,0,5]])
# A = R>0.5; A[A == True] = 1; A[A == False]   = 0; A = A.astype(np.float64, copy=False)

In [16]:
U,I,Rmse = runALS(R,A ,3, 40, 0.06)

Initiating 
Starting Iterations
Error after solving for User Matrix: 1.5135975338494259
Error after solving for Item Matrix: 0.9226465085506693
0th iteration is complete...
Error after solving for User Matrix: 0.8926472538504971
Error after solving for Item Matrix: 0.8835292690986901
1th iteration is complete...
Error after solving for User Matrix: 0.8779462389572084
Error after solving for Item Matrix: 0.8740459376874056
2th iteration is complete...
Error after solving for User Matrix: 0.8697313749669886
Error after solving for Item Matrix: 0.8659790160514612
3th iteration is complete...
Error after solving for User Matrix: 0.861412016569796
Error after solving for Item Matrix: 0.8575570103177342
4th iteration is complete...
Error after solving for User Matrix: 0.85342056000006
Error after solving for Item Matrix: 0.8503115675767781
5th iteration is complete...
Error after solving for User Matrix: 0.8473118669456599
Error after solving for Item Matrix: 0.8453375435213861
6th iteration

In [17]:
print(U.shape)
print(I.shape)
Rcap = U.dot(I)
print(Rcap)

(943, 3)
(3, 1682)
[[3.93198574 3.08712057 3.07152001 ... 1.3349548  2.93271786 2.66078362]
 [3.81984705 2.88893528 2.7160348  ... 1.701542   2.81393642 2.63207285]
 [3.51632477 3.06142582 3.08197581 ... 0.64985547 2.5385013  1.83470233]
 ...
 [4.375921   3.44202655 3.20470229 ... 1.78653115 3.16169576 2.71787087]
 [4.50547278 3.80539541 3.35422065 ... 1.70154785 3.0743887  2.07579213]
 [3.95021903 3.61269917 3.56130943 ... 0.54969909 2.76022941 1.64754302]]


In [18]:
import numpy as np
np.savetxt('Rcap.csv', Rcap, delimiter=',')

In [19]:
import numpy as np

# Load the matrix from the CSV file
loaded_matrix = np.loadtxt('/home/23m1526/100K/Rcap.csv', delimiter=',')
print(loaded_matrix)

[[3.93198574 3.08712057 3.07152001 ... 1.3349548  2.93271786 2.66078362]
 [3.81984705 2.88893528 2.7160348  ... 1.701542   2.81393642 2.63207285]
 [3.51632477 3.06142582 3.08197581 ... 0.64985547 2.5385013  1.83470233]
 ...
 [4.375921   3.44202655 3.20470229 ... 1.78653115 3.16169576 2.71787087]
 [4.50547278 3.80539541 3.35422065 ... 1.70154785 3.0743887  2.07579213]
 [3.95021903 3.61269917 3.56130943 ... 0.54969909 2.76022941 1.64754302]]


In [20]:
import numpy as np

def predRatings(user_item_matrix, Rcap, j, movies_df):
    def non_Rated_movie(user_item_matrix, j):
        zero_indices = np.where(user_item_matrix[:, j] == 0)[0]
        return zero_indices.tolist()

    def store_ratings_sorted(Rcap, zero_indices, j):
        ratings_dict = {}
        for i in zero_indices:
            rating = Rcap[i, j]
            ratings_dict[i] = rating
        sorted_ratings_dict = dict(sorted(ratings_dict.items(), key=lambda x: x[1], reverse=True))
        sorted_ratings_dict = dict(list(sorted_ratings_dict.items())[:20])
        return sorted_ratings_dict

    def get_movie_names(movies_df, top_20_ratings_dict):
        movie_names_dict = {}
        for key in top_20_ratings_dict.keys():
            movie_id = movies_df.loc[movies_df['movieId'] == key, 'title'].values[0]
            movie_names_dict[key] = movie_id
        return movie_names_dict

    zero_indices = non_Rated_movie(user_item_matrix, j)
    top_20_ratings_dict = store_ratings_sorted(Rcap, zero_indices, j)
    movie_names_dict = get_movie_names(movies_df, top_20_ratings_dict)

    return movie_names_dict

In [21]:
import pandas as pd
colName = ["movieId", "title", "Release Date", "Video Release Date",
           "IMDb URL", "Unknown", "Action", "Adventure", "Animation",
           "Children's", "Comedy", "Crime", "Documentary", "Drama", "Fantasy",
           "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi",
           "Thriller", "War", "Western"]

movies_df = pd.read_csv("u.item", sep="|", encoding="iso-8859-1",  names=colName)
movies_df.drop(columns= ["Release Date", "Video Release Date",
           "IMDb URL", "Unknown", "Action", "Adventure", "Animation",
           "Children's", "Comedy", "Crime", "Documentary", "Drama", "Fantasy",
           "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi",
           "Thriller", "War", "Western"], inplace=True)

In [22]:
movies_df

,movieId,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)
...,...,...
1677,1678,Mat' i syn (1997)
1678,1679,B. Monkey (1998)
1679,1680,Sliding Doors (1998)
1680,1681,You So Crazy (1994)


In [23]:
p = predRatings(R,Rcap,253,movies_df)

In [24]:
p

{126: 'Spitfire Grill, The (1996)',
 426: 'Transformers: The Movie, The (1986)',
 506: 'Rebel Without a Cause (1955)',
 339: 'Mad City (1997)',
 687: "McHale's Navy (1997)",
 849: 'Days of Thunder (1990)',
 810: 'Shadow, The (1994)',
 627: 'Robin Hood: Prince of Thieves (1991)',
 470: 'Tombstone (1993)',
 433: 'Heathers (1989)',
 811: 'Thirty-Two Short Films About Glenn Gould (1993)',
 848: 'Murder, My Sweet (1944)',
 619: 'Extreme Measures (1996)',
 211: 'M*A*S*H (1970)',
 906: 'Oscar & Lucinda (1997)',
 151: 'Willy Wonka and the Chocolate Factory (1971)',
 598: 'Big Squeeze, The (1996)',
 165: 'Jean de Florette (1986)',
 635: 'Fog, The (1980)',
 387: 'Age of Innocence, The (1993)'}

In [13]:
ep = [2, 4, 6, 8, 10, 12, 14, 16, 18, 20]
rmse_CALS = []

for ep in ep:
    rmse_CALS.append(Rmse)

print('The RMSE for different epsilons are', rmse_CALS)

The RMSE for different epsilons are [0.08052301958167798, 0.08052301958167798, 0.08052301958167798, 0.08052301958167798, 0.08052301958167798, 0.08052301958167798, 0.08052301958167798, 0.08052301958167798, 0.08052301958167798, 0.08052301958167798]


In [1]:
import csv

rmse_CALS =  [0.8301938214881543, 0.8301938214881543, 0.8301938214881543, 0.8301938214881543,
  0.8301938214881543, 0.8301938214881543, 0.8301938214881543, 0.8301938214881543, 0.8301938214881543, 0.8301938214881543]


csv_file_path = '/home/23m1526/output2.csv'



In [2]:
with open(csv_file_path, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(['List Name', 'Values'])
    writer.writerow(['rmse_CALS'] + rmse_CALS)
